In [1]:
import numpy as np
import scipy.stats.distributions as dist
import scipy.stats as stats
import statsmodels.stats as sts
import statsmodels.api as sm

# Difference between two proportions

In [54]:
def get_p_value_and_t_stat (count1, nobs1, count2, nobs2):
    # Estimates of the population proportions
    p1 = round(count1 / nobs1, 6)
    p2 = round(count2 / nobs2, 6)

    # Estimate of the combined population proportion
    phat = (count1 + count1) / (nobs1 + nobs2)

    # Estimate of the variance of the combined population proportion
    va = phat * (1 - phat)

    # Estimate of the standard error of the combined population proportion
    se = np.sqrt(va * (1 / nobs1 + 1 / nobs2))

    # Test statistic and its p-value
    test_stat = (p1 - p2) / se
    pvalue = 2*dist.norm.cdf(-np.abs(test_stat))

    # Print the test statistic its p-value
    return test_stat, pvalue


def conf_intervals_proportion_diff (count1, nobs1, count2, nobs2, alpha=0.05):
    # Estimates of the population proportions
    p1 = round(count1 / nobs1 , 6)
    p2 = round(count2 / nobs2, 6)
    
    diff = round(p1 - p2,6)

    se_1 = np.sqrt(p1 * (1 - p1)/ nobs1)
    se_2 = np.sqrt(p2 * (1 - p2)/ nobs2)
    se_diff = np.sqrt(se_1**2 + se_2**2)

    # z critical value
    confidence = 1 - alpha
    z = stats.norm(loc = 0, scale = 1).ppf(confidence + alpha / 2)
    
    moe = round(z * se_diff,6)
    
    lcb = diff - moe
    ucb = diff + moe
    
    result_as_string = "{} += {}".format(diff,moe)
    
    return diff, moe, lcb, ucb


def conf_intervals_one_proportion(count1, nobs1, alpha=0.05):
    
    p1 = round(count1 / nobs1, 6)
    se = np.sqrt(p1 * (1 - p1)/ nobs1)
    
    # z critical value
    confidence = 1 - alpha
    z = stats.norm(loc = 0, scale = 1).ppf(confidence + alpha / 2)
    
    moe = round(z * se,6)
    
    return moe

In [57]:
# input Data

# Sample sizes (Visitors)
n_1 = 5976004  
n_2 = 5972203   

# Proportions (Visitors with clicks)
clicks_1 = 2001183
clicks_2 = 2004354

# Confidencce level
alpha=0.1 

In [69]:
# Conversion rate (CR)
cr_1 = round(clicks_1/n_1,6)
cr_2 = round(clicks_2/n_2,6)
cr_diff = round(cr_2-cr_1,6) # abs differnce in ET tool
cr_diff_relative = round(cr_diff/cr_1,6) # relative differenc in ET tool

# Conf. intervals for each CR
moe_1 = conf_intervals_one_proportion(clicks_1, n_1, alpha) 
moe_2 = conf_intervals_one_proportion(clicks_2, n_2, alpha) 

# Caclulate conf intervals directly
ci_1 = sm.stats.proportion_confint(clicks_1, n_1, alpha)
ci_2 = sm.stats.proportion_confint(clicks_2, n_2, alpha)

print("CR_1: {}+={} ({})".format(cr_1, moe_1, ci_1))
print("CR_2: {}+={} ({})".format(cr_2, moe_2, ci_2))
print(cr_diff)
print(ci_diff_relative)

CR_1: 0.33487+=0.000318 ((0.3345522050375604, 0.3351873065156447))
CR_2: 0.335614+=0.000318 ((0.33529601643224166, 0.3359316695657058))
0.000744
0.002222


In [70]:
# calculate statistical significance (t_stats and p value) and conf. intervals for the difference

# 1st way
t_stat, p_value = get_p_value_and_t_stat(clicks_2, n_2, clicks_1 ,n_1)
print(p_value) # significance in ET tool

# 2nd way
# Create populations 
pop_1 = np.array([1]*clicks_1+[0]*(n_1-clicks_1))
pop_2 = np.array([1]*clicks_2+[0]*(n_2-clicks_2))
# calulate p_value based on arrays 
t_stat, p_value = stats.ttest_ind(pop_2,pop_1)
print(p_value) # significance in ET tool

if p_value < 0.05:
    print('p-value is small, the difference is stat. significant')
else:
    print('p-value is large, the difference is not stat. significant')

0.00646299278272296
0.006446317007577497
p-value is small, the difference is stat. significant


In [93]:
# Confidence intervals

# 1 way

# absolute difference
diff, moe, lcl, ucl = conf_intervals_proportion_diff(clicks_2, n_2, clicks_1, n_1)
print("CR difference: {}+={} ({}, {})".format(diff, moe, lcl, ucl))

# relative difference
print("CR difference(relative): {}+={} ({}, {})".format(round(diff/cr_1,6), round(moe/cr_1,6), 
                                                        round(lcl/cr_1,6), round(ucl/cr_1,6))
     )



# 2 way
      
# absolute difference
print("using API")
ci_diff = sts.proportion.confint_proportions_2indep(clicks_2, n_2, clicks_1, n_1)
print("CR difference: {}".format(ci_diff))

# relative difference
lcl_rel = round(ci_diff[0]/cr_1,6) 
ucl_rel = round(ci_diff[1]/cr_1,6)
print("CR difference(relative): ({},{})".format(lcl_rel,ucl_rel))

CR difference: 0.000744+=0.000535 (0.00020899999999999998, 0.001279)
CR difference(relative): 0.002222+=0.001598 (0.000624, 0.003819)
using API
CR difference: (0.00020873741160846407, 0.0012794365818959554)
CR difference(relative): (0.000623,0.003821)


# Questions

1. Is there a library that calculates p_value based on sample size and # of observations for Proportion difference?I.e. Similar to this function - get_p_value_and_t_stat(count1, nobs1, count2, nobs2)

## Learnings

1. Bigger sample size(traffic) -> better (faster to detect significance)
2. Higher  proportion levels (or CR) - better (faster) to detect significance